# Loads and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Download and Import Latent Space data and corresponding Original Data from S3

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/treated0_l.p ../../temp/ldata_knn/treated0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/ldata/m6_knn_no_cluster_only_latent/match0_l.p ../../temp/ldata_knn/match0_l.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/treated0_o.p ../../temp/odata_knn/treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/odata/m6_knn_no_cluster_only_latent/match0_o.p ../../temp/odata_knn/match0_o.p

In [4]:
import pickle
with open(path+'/ldata_knn/match0_l.p', 'rb') as fp:
    controls_sick_latent = pickle.load(fp)
with open(path+'/ldata_knn/treated0_l.p', 'rb') as fp:
    treated_sick_latent = pickle.load(fp)
with open(path+'/odata_knn/match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_knn/treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [5]:
controls = controls_sick_latent[controls_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated = treated_sick_latent[treated_sick_latent.PERS_ID>"0"].sort_values(by=["PERS_ID","MYR"])

In [6]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

$$d = |\frac{(\bar{x_t}-\bar{x_c})}{\sqrt{\frac{s_t^2 + s_c^2}{2}}}| $$

In [7]:
def ASMD_cont_calculate(tf, cf):
    t_mean = np.mean(tf.values)
    c_mean = np.mean(cf.values)
    t_var = np.var(tf.values)
    c_var = np.var(cf.values)
    d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
    return d

In [8]:
def ASMD_binary_calculate(tf, cf):
    pt = np.mean(tf.values)
    pc = np.mean(cf.values)
    d = np.abs((pt-pc)/(pt*(1-pt)+pc*(1-pc)))
    return d
    

In [9]:
def ASMD_final(cols, col_cat, cdf, tdf):
    ASMD = pd.DataFrame()
    cf = cdf[cols]
    tf = tdf[cols]
    for i, (col_cat_each, col_name_each) in enumerate(zip(col_cat, cols)):
        if col_cat_each == 'c':
            d = ASMD_cont_calculate(tf[col_name_each], cf[col_name_each])
        else:
            d = ASMD_binary_calculate(tf[col_name_each], cf[col_name_each])
        ASMD.loc[0, col_name_each] = d 
    return ASMD
    

### Latent Space

In [10]:
latent_cols = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16']

In [11]:
from itertools import repeat
given_value ='c'
latent_cols_cat=[]
latent_cols_cat.extend(repeat(given_value,len(latent_cols)))

In [12]:
ASMD = ASMD_final(latent_cols, latent_cols_cat, controls, treated)
ASMD

x1        x2        x3        x4        x5        x6        x7        x8        x9       x10       x11       x12       x13       x14       x15       x16
0  0.007234  0.010884  0.014371  0.014107  0.025718  0.009188  0.012039  0.007646  0.010025  0.015712  0.014542  0.006073  0.011024  0.008754  0.009017  0.002524

### Original Space with George's Features only

In [13]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']
george_cols_cat = ['c', 'b', 'c', 'b', 'b', 'b', 'b']

In [14]:
ASMD = ASMD_final(george_cols, george_cols_cat, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.041522  0.365037   0.060294    0.041446    0.289862         0.090942        0.120924

### Original Space with all the features

In [15]:
original_cols = controls_orig.columns.to_list()[2:]

In [16]:
l1 = ['c', 'b', 'b', 'c', 'c', 'c']

given_value ='b'

orig_CC_cols_cat=[]
orig_CC_cols_cat.extend(repeat(given_value,len(original_cols)-len(l1)))

original_cols_cat = l1 + orig_CC_cols_cat

In [17]:
ASMD = ASMD_final(original_cols, original_cols_cat, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [18]:
ASMD.T

0
AGE_AT_MIDMONTH             0.041522
MBR_GNDR                    0.365037
MEDICAID_LOB                0.495823
ALLOW_AMT                   0.060294
ALLOW_ER                    0.018635
ALLOW_IP                    0.026081
CC_ADHD                     0.231257
CC_ALZHEIMER                1.001197
CC_ALZHEIMER_PLUS           0.786323
CC_AMI                      0.459559
CC_ANEMIA                   0.045395
CC_ANXIETY_DISORDER         0.068134
CC_ARTHRITIS                0.092292
CC_ASTHMA                   0.108492
CC_ATRIAL_FIB               0.108036
CC_AUTISM                   1.001492
CC_BIPOLAR                  0.031770
CC_BPH                      0.229121
CC_BREAST_CANCER            0.390413
CC_CATARACT                 0.067191
CC_CEREBRAL_PALSY           0.104815
CC_CHRONIC_KIDNEY           0.004873
CC_COLORECTAL_CANCER        0.132608
CC_COPD                     0.001472
CC_CYSTIC_FIBROSIS          0.537986
CC_DEPRESSION               0.034875
CC_DEPRESSIVE_DISORDERS     0.139234
CC_DEVELOP_DELAYS           0.318113
CC_DIABETES                 0.053408
CC_DISAB_DX_CNT             0.002627
CC_DX_CNT                   0.039794
CC_ENDOMETRIAL_CANCER       0.130357
CC_EPILEPSY                 0.095993
CC_FIBROMYALGIA             0.028201
CC_GLAUCOMA                 0.145952
CC_HEARING_IMPAIR           0.087750
CC_HEART_FAILURE            0.019411
CC_HEPATITIS_A              1.000269
CC_HEPATITIS_B_ACUTE        0.548076
CC_HEPATITIS_B_CHRONIC      0.058113
CC_HEPATITIS_C_ACUTE        1.000407
CC_HEPATITIS_C_CHRONIC      0.591468
CC_HEPATITIS_C_UNSPECIFIED  0.345632
CC_HEPATITIS_D                   NaN
CC_HEPATITIS_E                   NaN
CC_HEPATITIS_GEN            0.396342
CC_HIP_FRACTURE             1.000251
CC_HYPERLIPIDEMIA           0.045125
CC_HYPERTENSION             0.090942
CC_HYPOTHYROID              0.120924
CC_INT_DISAB                0.367991
CC_ISCHEMIC                 0.032562
CC_LEARN_DISAB              0.094310
CC_LEUKEMIAS                0.303422
CC_LIVER                    0.055420
CC_LUNG_CANCER              0.200884
CC_MIGRAINE                 0.203318
CC_MOBILITY_IMPAIR          0.120312
CC_MULTIPLE_SCLEROSIS       0.251022
CC_MUSCULAR_DYSTROPHY            NaN
CC_OBESITY                  0.289862
CC_OSTEOPOROSIS             0.027901
CC_PERIPHERAL               0.040450
CC_PERSONALITY              0.086976
CC_PRESSURE                 0.098125
CC_PROSTATE_CANCER          0.119239
CC_PTSD                     0.189812
CC_SCHIZOPHRENIA            0.242924
CC_SCHIZOPHRENIA_OTHER      0.365328
CC_SPINAL_CORD              0.324728
CC_SPINA_BIFIDA             0.278666
CC_STROKE                   0.079121
CC_TOBACCO                  0.041446
CC_TRAUMATIC_BRAIN          0.247431
CC_VISUAL_IMPAIR            0.023666